In [1174]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [1175]:
faa = pd.read_csv("data/FAA/FAA_WEB-Report-22188.csv")
bls_oak = pd.read_excel("data/BLS/SeriesReport-OAK-HWD-BKL.xlsx", skiprows=10)
bls_sf = pd.read_excel("data/BLS/SeriesReport-SF-RWC-SSF.xlsx", skiprows=10)
bls_sj = pd.read_excel("data/BLS/SeriesReport-SJ-SVL-SNC.xlsx", skiprows=10)
covid_h = pd.read_csv("data/CitySF/COVID-19_Hospitalizations_Over_Time_20240830.csv")
covid_v = pd.read_csv("data/CitySF/ARCHIVED__COVID-19_Vaccinations_Given_to_SF_Residents_Over_Time_20240830.csv")
pop = pd.read_csv("data/FRED/CASANF0POP.csv")
pba2050 = pd.read_csv("data/PBA2050/PBA_growth_profiles_normalized_2022.csv")
taf2023 = pd.read_excel("data/FAA/AirportsOperations_SFO.xlsx")

**Subset FAA data for SFO commercial operations only**

In [1176]:
faa_sfo = faa.copy().loc[faa.Airport=="SFO"]
faa_sfo['Commercial_Ops'] = faa_sfo['AirCarrier']+faa_sfo['AirTaxi']
faa_sfo = faa_sfo.copy()[['Year','Commercial_Ops']]

**Aggregate BLS data for three metropolitan areas for total Bay Area employment**

In [1177]:
def bls_avg(bls):
    '''
    Calculates average employment count for each year of BLS data.
    '''
    bls_melt = bls.melt(id_vars=['Year'], var_name='Month', value_name='Employment')
    bls_avg = bls_melt.groupby(['Year']).mean().reset_index()
    
    return bls_avg

In [1178]:
bls_oak_avg = bls_avg(bls_oak)
bls_sf_avg = bls_avg(bls_sf)
bls_sj_avg = bls_avg(bls_sj)
bls_all = bls_oak_avg.merge(bls_sf_avg, on='Year')
bls_all = bls_all.merge(bls_sj_avg, on='Year')
bls_all['Employment'] = bls_all['Employment_x']+bls_all['Employment_y']+bls_all['Employment']
bls_all.drop(columns=['Employment_x','Employment_y'], inplace=True)

**Aggregate COVID-19 hospitalizations and vaccinations by year**

In [1179]:
covid_h['reportdate'] = pd.to_datetime(covid_h['reportdate'], format="%Y/%m/%d")
covid_h['Year'] = covid_h['reportdate'].apply(lambda x:x.year)
covid_h['patientcount'] = covid_h['patientcount']/covid_h['hospitalreportingpct']
covid_h_sum = covid_h.groupby(['Year']).sum()[['patientcount']].reset_index()
covid_h_sum.rename(columns={'patientcount':'covid_h'}, inplace=True)

In [1180]:
covid_v = covid_v.loc[covid_v['administering_provider_type']=="All Providers"]
covid_v['date_administered'] = pd.to_datetime(covid_v['date_administered'], format="%Y/%m/%d")
covid_v['Year'] = covid_v['date_administered'].apply(lambda x:x.year)
covid_v_avg = covid_v.groupby(['Year']).mean()[['cumulative_recipients']].reset_index()
covid_v_avg.rename(columns={'cumulative_recipients':'covid_v'}, inplace=True)

**Reformat population data**

In [1181]:
pop['DATE'] = pd.to_datetime(pop['DATE'], format="%Y-%m-%d")
pop['Year'] = pop['DATE'].apply(lambda x:x.year)
pop['Pop'] = pop['CASANF0POP']*1000
pop = pop[['Year','Pop']]

**Merge datasets and output correlations with Commercial_Ops**

In [1182]:
data = faa_sfo.merge(bls_all, on='Year')
data = data.merge(covid_h_sum, on='Year', how='left')
data = data.merge(covid_v_avg, on='Year', how='left')
data = data.merge(pop, on='Year', how='left')
data = data.fillna(0)

**Create new variable for non-vaccinated population**

In [1183]:
data['non_vax_pop'] = data['Pop']-data['covid_v']
data['non_vax_pop'] = data.apply(lambda x:0 if x.Year<2020 else x.non_vax_pop, axis=1)
data['non_vax_pop'] = data['non_vax_pop'].apply(lambda x:0 if x<0 else x)
data.drop(columns=['covid_v','Pop'], inplace=True)

**Add dummy variable for post-9/11 effects and output correlations to Commercial_Ops**

In [1184]:
data['dummy_911'] = data.apply(lambda x:1 if x.Year in [2001,2002,2003] else 0, axis=1)

In [1185]:
data['dummy_covid'] = data.apply(lambda x:1 if x.Year in [2020,2021] else 0, axis=1)

**Output correlations of variables with Commercial_Ops**

In [1186]:
data.corr()[['Commercial_Ops']]

,Commercial_Ops
Year,-0.140069
Commercial_Ops,1.000000
Employment,0.084124
covid_h,-0.593244
non_vax_pop,-0.669679
dummy_911,-0.257498
dummy_covid,-0.714581


**Correlations for data prior to 2019 only**

In [1187]:
data_2019 = data.loc[data.Year<=2019]
data_2019.corr()[['Commercial_Ops']]

,Commercial_Ops
Year,0.313257
Commercial_Ops,1.000000
Employment,0.589422
covid_h,NaN
non_vax_pop,NaN
dummy_911,-0.484711
dummy_covid,NaN


**Correlations for data prior to 2019 and not including 2001-2003**

In [1188]:
data_normal_yr = data_2019.loc[~data.Year.isin([2001,2002,2003])]
data_normal_yr.corr()[['Commercial_Ops']]

,Commercial_Ops
Year,0.32121
Commercial_Ops,1.00000
Employment,0.65008
covid_h,NaN
non_vax_pop,NaN
dummy_911,NaN
dummy_covid,NaN


**Create train/test sets, use years after 2000 (results in better regression)**

In [1189]:
data = data.loc[data.Year >= 2000]

In [1190]:
test_years = [1991,1994,1996,1999,2002,2005,2008,2011,2014,2017,2022]
data_test = data.loc[data.Year.isin(test_years)]
data_train = data.loc[~data.Year.isin(test_years)]
X_cols = ['Year','Employment','covid_h','non_vax_pop','dummy_911','dummy_covid']
y_col = ['Commercial_Ops']

In [1191]:
X = data[X_cols]
y = np.ravel(data[y_col])
X_train = data_train[X_cols]
X_test = data_test[X_cols]
y_train = np.ravel(data_train[y_col])
y_test = np.ravel(data_test[y_col])

**Train and score regression model**

In [1192]:
reg = LinearRegression().fit(X_train,y_train)
print("R2 on train set: {}".format(round(reg.score(X_train,y_train),3)))
print("R2 on test set: {}".format(round(reg.score(X_test,y_test),3)))

R2 on train set: 0.988
R2 on test set: 0.976


**Plot actual vs. predicted operations**

In [1193]:
y_pred = reg.predict(X)
y_df = pd.DataFrame()
y_df['Year'] = data['Year']
y_df['Commercial_Ops_actual'] = y
y_df['Commercial_Ops_predicted'] = y_pred
y_df = y_df.melt(id_vars=['Year'], value_name='Operations', var_name='Variable')
y_df.rename(columns={'value':'Operations'}, inplace=True)
fig = px.line(y_df, x='Year', y='Operations', color='Variable')
fig.show()

**Use PBA2050 employment projections for future data to fit model predictions**

In [1194]:
totemp = pba2050.loc[(pba2050.variable=="TOTEMP") & (pba2050.cnty_abbr=="TOT")]
totemp.drop(columns=['gpf_id','variable','cnty_abbr'], inplace=True)
for col in totemp.columns:
    totemp[col] = totemp[col]/totemp['2023'].values[0]
totemp = totemp.melt(var_name="Year",value_name="growth")
totemp['Year'] = totemp['Year'].apply(lambda x:int(x))
totemp = totemp.loc[totemp.Year >= 2023]

**Set COVID hospitalizations for 2024 to 16000 (5154 hospitalizations in first 4 months of year), assume 50% decrease in hospitalizations each subsequent year**

In [1195]:
data_fc = pd.DataFrame(columns=data.columns)
data_fc['Year'] = np.linspace(2024,2050,2050-2024+1)
data_fc['covid_h'] = [int(16000*0.5**n) for n in range(0,2050-2024+1)]
data_fc['non_vax_pop'] = 0
data_fc['dummy_911'] = 0
data_fc['dummy_covid'] = 0
emp_2023 = data.loc[data['Year'] == 2023]['Employment'].values[0]
data_fc = data_fc.merge(totemp, on=['Year'], how='left')
data_fc['Employment'] = emp_2023*data_fc['growth']
data_fc.drop(columns=['Commercial_Ops','growth'], inplace=True)
data_fc = data_fc[X_cols]

**Use regression model to predict operations from 2024-2050**

In [1196]:
y_proj = reg.predict(data_fc)
y_df_proj = pd.DataFrame()
y_df_proj['Year'] = data_fc['Year']
y_df_proj['Commercial_Ops_actual'] = np.nan
y_df_proj['Commercial_Ops_predicted'] = y_proj
y_df_proj = y_df_proj.melt(id_vars=['Year'], value_name='Operations', var_name='Variable')
y_df_all = pd.concat([y_df, y_df_proj])

**Set upper bound on predictions based on SFO capacity (506,600 total operations)**

In [1197]:
taf_sfo = taf2023.loc[taf2023.ayear >= 2000]
taf_sfo['Total'] = taf_sfo[['itn_Ac','itn_at','itn_ga','itn_mil']].sum(axis=1)
taf_sfo['itn_mil'] = taf_sfo['itn_mil'].apply(lambda x:2400 if x>2400 else x)
taf_sfo['itn_ga'] = taf_sfo['itn_ga'].apply(lambda x:12900 if x>12900 else x)
taf_sfo['comm_ops_max'] = 506600-taf_sfo['itn_mil']-taf_sfo['itn_ga']
taf_sfo.rename(columns={'ayear':'Year'}, inplace=True)
y_df_all = y_df_all.merge(taf_sfo[['Year','comm_ops_max']], on=['Year'], how='left')
y_df_all['Operations'] = y_df_all.apply(lambda x:x.comm_ops_max if x.Operations>x.comm_ops_max else x.Operations, axis=1)
y_df_all.drop(columns=['comm_ops_max'], inplace=True)
fig = px.line(y_df_all, x='Year', y='Operations', color='Variable')
fig.show()

**Output projection to send to CARB**

In [1198]:
sfo_carb_2022 = taf2023.loc[taf2023.ayear.isin([2022,2023])]
sfo_carb = y_df_all.loc[y_df_all.Variable=="Commercial_Ops_predicted"][['Year','Operations']]
sfo_carb = sfo_carb.loc[sfo_carb.Year>=2024]
sfo_carb = sfo_carb.merge(taf_sfo, on='Year', how='left')
sfo_carb['itn_at'] = sfo_carb['itn_at'].apply(lambda x:21100 if x>21100 else x)
sfo_carb['itn_Ac'] = sfo_carb['Operations']-sfo_carb['itn_at']
sfo_carb['itn_Ac'] = sfo_carb['itn_Ac'].apply(lambda x:int(x))
sfo_carb.drop(columns=['Operations','comm_ops_max'], inplace=True)
sfo_carb.rename(columns={'Year':'ayear'}, inplace=True)
sfo_carb = sfo_carb[['locid','scenario','ayear','itn_Ac', 'itn_at', 'itn_ga','itn_mil', 'loc_ga', 'loc_mil', 'tot_overs']]
sfo_carb = pd.concat([sfo_carb_2022, sfo_carb])
sfo_carb.to_excel("output/SFO_projection_to_CARB.xlsx", index=False)

**Output forecast profile for inventory**

In [1199]:
sfo_forecast = y_df_all.loc[y_df_all.Year>=2022]
sfo_forecast = sfo_forecast.loc[sfo_forecast.Variable=="Commercial_Ops_predicted"]
sfo_forecast['Normalized'] = sfo_forecast['Operations']/sfo_forecast.loc[sfo_forecast.Year==2022]['Operations'].values[0]
sfo_forecast['Operations'] = sfo_forecast['Operations'].apply(lambda x:int(x))
sfo_forecast.drop(columns=['Variable'], inplace=True)
sfo_forecast.to_csv("output/SFO_forecast_profile.csv", index=False)